# Language guessing

In [14]:
import langid
langid.set_languages(["de", "en"])

def guess_language(string_in_language):
    return (langid.classify(string_in_language)[0])

# print(langid.classify("Zur transgenerationalen Traumatisierung"))
# print(langid.classify("Ätiologie und Ansätze für die Therapie"))

# print(langid.classify("Zur transgenerationalen Traumatisierung")[0])
print(guess_language("\"A 'true' artist may draw mountainous seas!\" - Eine Würdigung von Paul Watzlawick zu seinem 100. Geburtstag"))
# print(guess_language("What does it mean for children's development whether and to what extent both parents are employed and they accordingly spend part of the day outside the family? And how should care for young children be structured? Research findings from Developmental Psychology provide some answers. (translated by DeepL)"))

de


# Checking strings for non-letters

In [12]:

language = "EnglishX$X$"
# check if string contains any non-letter character:
if not language.isalpha():
    print("yes")
else:
    print("no")

yes


# Reconciling affiliation with ror

In [15]:
import requests_cache
from datetime import timedelta
# from mappings import geonames_countries
# from mappings import abstract_origins
# from mappings import dd_codes

ROR_API_URL = "https://api.ror.org/organizations?affiliation="

from mappings import dd_codes

def replace_encodings(text):
    for case in dd_codes:
        text = text.replace(case[0], case[1]) 
    return text

urls_expire_after = {
    # Custom cache duration per url, 0 means "don't cache"
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/09183&lang=de': 0,
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/': 0,
}

session = requests_cache.CachedSession(
    ".cache/requests",
    allowable_codes=[200, 404],
    expire_after=timedelta(days=30),
    urls_expire_after=urls_expire_after,
)

def get_ror_id_from_api(affiliation_string):
    # this function takes a string with an affiliation name and returns the ror id for that affiliation from the ror api
    # clean the string to make sure things like "^DDS" are replaced:
    affiliation_string = replace_encodings(affiliation_string)
    #replace_encodings(affiliation_string)
    ror_api_url = ROR_API_URL + affiliation_string
    # make a request to the ror api:
    # ror_api_request = requests.get(ror_api_url)
    # make request to api with caching:
    ror_api_request = session.get(
            ror_api_url, timeout=20
    )
    # if the request was successful, get the json response:
    if ror_api_request.status_code == 200:
        ror_api_response = ror_api_request.json()
        # check if the response has any hits:
        if len(ror_api_response["items"]) > 0:
            # if so, get the item with a key value pair of "chosen" and "true" and return its id:
            for item in ror_api_response["items"]:
                if item["chosen"] == True:
                    return item["organization"]["id"]
        else:
            return None
    else:
        return None
    

    # here is a list of affiliation strings to go through:
affiliation_strings = [
    "Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Sektion Medizinische Psychologie, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Psychology School, Hochschule Fresenius ^DDS University of Applied Sciences, Düsseldorf",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Department of Child and Adolescent Psychiatry, Psychosomatics and Psychotherapy; LVR Klinikum Essen; University Hospital Essen; University of Duisburg-Essen; Essen",
    "Child and Adolescent Psychiatry/Psychology, Erasmus Medical Center Rotterdam"
]

# use the function to get the ror id for each affiliation string:

# for affiliation_string in affiliation_strings:
#     print(replace_encodings(affiliation_string) + ": " + str(get_ror_id_from_api(affiliation_string)))

# print(replace_encodings("Stimulus ^DDS non psychological interference ^DDL analogy"))
print(replace_encodings("Zagreb Children^D&gt;'s Hospital, Pediatric Clinic"))
print("Test " + replace_encodings("Zagreb Children^D&gt;'s Hospital, Pediatric Clinic") + " more")
print(replace_encodings("p &lt; .05"))
print(replace_encodings(' &lt; '))
print(replace_encodings('haha ^DIF ^DTM'))
print(replace_encodings('Geo^Dffrey Je^Dfferson'))
# print(get_ror_id_from_api("Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm"))

Zagreb Childrenğt;'s Hospital, Pediatric Clinic
Test Zagreb Childrenğt;'s Hospital, Pediatric Clinic more
p &lt; .05
 &lt; 
haha ∞ ™
Geoffrey Jefferson
